In [2]:
import sys
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras import backend as K
from keras.layers import Activation, concatenate, Conv1D, Dense, Dropout, Flatten, Input, Lambda
from keras.models import Model
from generator import AudioGenerator, kltls, labels_to_ys, ys_to_labels
import pickle
import numpy as np
from time import time as timestamp
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt

# Allows me to import my modules
sys.path.append('./modules')
from audio_utils import *

Using TensorFlow backend.


Attempting to read settings file...
	Read successfully!


In [3]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6770255697314423231, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 577778483
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13278293324490974531
 physical_device_desc: "device: 0, name: GeForce GTX 650, pci bus id: 0000:01:00.0, compute capability: 3.0"]

In [4]:
# Tells Tensorflow to use the GPU
config = tf.ConfigProto(allow_soft_placement=True,
                        device_count = {'CPU' : 1,
                                        'GPU' : 1},
                        log_device_placement = True
                       )

session = tf.Session(config=config)
K.set_session(session)

In [5]:
# Data generators
generators = {"training": None, "test": None}
N = {"training": 0, "test": 0}
for data_type in generators.keys():
    sample_metadata = get_file_classes(data_type)
    N[data_type] = len(sample_metadata)
    filenames = [sm["filepath"] for sm in sample_metadata]
    labels = [sm["labels"] for sm in sample_metadata]
    generators[data_type] = AudioGenerator(filenames, labels, data_type)

In [6]:
batch_0 = generators["training"].__getitem__(0)
print("In shape:", batch_0[0].shape, "\nOut shape:", batch_0[1].shape)

In shape: (100, 12000, 1) 
Out shape: (100, 10)


In [7]:
# Test whether generator arguments are picklable (whether they can be multiprocessed)
use_multiprocessing = True
for gen in generators:
    try:
        pickle.dumps(gen)
    except:
        print(sys.exc_info())
        use_multiprocessing = False
        break
print("Picklable:", use_multiprocessing)

Picklable: True


In [8]:
# Adapted from https://keras.io/layers/writing-your-own-keras-layers/
def InceptionModule(model):
    # Skip connection (uses input in concat)
    skip = Lambda(lambda x: x)(model)
    # Size 1 kernel conv of input (with tanh activation)
    conv_1_tower = Conv1D(filters=128, kernel_size=1, strides=1, padding="valid", activation="tanh")(model)
    # Size 1 -> size 3 kernel conv of input (with tanh activation)
    conv_3_tower = Conv1D(filters=1, kernel_size=1, strides=1, padding="valid")(model)
    conv_3_tower = Conv1D(filters=128, kernel_size=3, strides=1, padding="causal", activation="tanh")(conv_3_tower)
    # Size 1 -> size 5 kernel conv of input (with tanh activation)
    conv_5_tower = Conv1D(filters=1, kernel_size=1, strides=1, padding="valid")(model)
    conv_5_tower = Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="tanh")(conv_5_tower)
    # Size 1 -> size 7 kernel conv of input (with tanh activation)
    conv_7_tower = Conv1D(filters=1, kernel_size=1, strides=1, padding="valid")(model)
    conv_7_tower = Conv1D(filters=128, kernel_size=7, strides=1, padding="causal", activation="tanh")(conv_7_tower)
    # Concatenate all activation images
    return concatenate([skip, conv_1_tower, conv_3_tower, conv_5_tower, conv_7_tower], axis=2)

In [9]:
# Reusable dilated convolution / inception module / dropout layer
def DilatedInceptionModuleLayer(model, drop_rate):
    model = Conv1D(filters=128, kernel_size=1, padding="causal", dilation_rate=2)(model)
    model = InceptionModule(model)
    return Dropout(rate=drop_rate)(model)

dim_rates = [0.1, 0.15, 0.2, 0.25, 0.3]

# Structure
"""
Rationale: 

3 "CausalConvAct" convolution layers which reduce the size of the sample space while increasing the size of the convolution space.
- Providing downscaling
(Feature extraction, while preserving temporal relationships)

Then "DilatedInceptionModule" layers which retain the size of the sample space while extracting more features.

- Using Convolutions to downsample from LeNet (?)
- Dropout paper
- ResNet for skip connections
- Inception module adapted from GoogLeNet
- Causal convolutions from WaveNet
"""
data = Input(shape=(12000, 1))
cnn = Conv1D(filters=32, kernel_size=7, strides=3, padding="causal", dilation_rate=1, activation="tanh")(data)
cnn = Conv1D(filters=64, kernel_size=7, strides=2, padding="causal", dilation_rate=1, activation="tanh")(cnn)
cnn = Conv1D(filters=128, kernel_size=5, strides=2, padding="causal", dilation_rate=1, activation="tanh")(cnn)
cnn = Dropout(rate=0.1)(cnn)
for drop_rate in dim_rates:
    cnn = DilatedInceptionModuleLayer(cnn, drop_rate)
cnn = Flatten()(cnn)
cnn = Dense(10, activation='sigmoid')(cnn)
model = Model(inputs=data, outputs=cnn)

for layer in model.layers:
    print(layer.name, layer.output_shape)

# Tensorboard logs
tb_logs = TensorBoard(log_dir="logs/{}".format(timestamp()))

# Compile with stocastic gradient descent and mean squared error loss (same as multilabelled paper)
model.compile(optimizer="sgd", loss="mean_squared_error")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
input_1 (None, 12000, 1)
conv1d_1 (None, 4000, 32)
conv1d_2 (None, 2000, 64)
conv1d_3 (None, 1000, 128)
dropout_1 (None, 1000, 128)
conv1d_4 (None, 1000, 128)
conv1d_6 (None, 1000, 1)
conv1d_8 (None, 1000, 1)
conv1d_10 (None, 1000, 1)
lambda_1 (None, 1000, 128)
conv1d_5 (None, 1000, 128)
conv1d_7 (None, 1000, 128)
conv1d_9 (None, 1000, 128)
conv1d_11 (None, 1000, 128)
concatenate_1 (None, 1000, 640)
dropout_2 (None, 1000, 640)
conv1d_12 (None, 1000, 128)
conv1d_14 (None, 1000, 1)
conv1d_16 (None, 1000, 1)
conv1d_18 (None, 1000, 1)
lambda_2 (None, 1000, 128)
conv1d_13 (None, 1000, 128)
conv1d_15 (None, 1000, 128)
conv1d_17 (None, 1000, 128)
conv1d_19 (None, 1000, 128)
concatenate_2 (None, 1000, 640)
dropout_3 (None, 1000, 640)
conv1d_20 (None, 1000, 128)
conv1d_22 (None, 1000, 1)
conv1d_24 (None, 1000, 

In [12]:
# Train model
batch_size = 100
epochs = 20
training_history = model.fit_generator(
                generator = generators["training"],
                steps_per_epoch = N["training"] // batch_size,
                validation_data = generators["test"],
                validation_steps = N["test"] // batch_size,
                max_queue_size=5,
                epochs = epochs,
                callbacks = [tb_logs],
                #use_multiprocessing = use_multiprocessing,
                #workers = 4
)

Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor with shape[200,500,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv1d_4/convolution/SpaceToBatchND}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss/mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
plt.figure()
xs = np.arange(0, epochs)
plt.plot(xs training_history.history["loss"], label="T loss")
plt.plot(xs, training_history.history["val_loss"], label="V loss")
plt.plot(xs, training_history.history["acc"], label="T acc")
plt.plot(xs, training_history.history["val_acc"], label="V acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

In [70]:
model_json = model.to_json()
with open("modelA2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modelA2.h5")
print("Saved model to disk")

Saved model to disk
